# **IMPORTS**

In [1]:
!pip install -U transformers
!pip install -U accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 67.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 8.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, pipeline
import torch
from transformers import BitsAndBytesConfig
import pandas as pd
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

2024-09-27 06:01:02.768748: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 06:01:02.768855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 06:01:02.897511: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
if torch.cuda.is_available():
    print('Automatic Mixed Precision (AMP) is supported.')
else:
    print('Automatic Mixed Precision (AMP) is not supported.')


Automatic Mixed Precision (AMP) is supported.


In [4]:
!nvidia-smi

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Fri Sep 27 06:01:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             27W /  250W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# **MODEL *(Gemma-2-2b-it)***

In [5]:
access_token = user_secrets.get_secret("hf_token")

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=access_token)


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# Answer style
<h3>Прежде, чем нагенерировать ответы на инструкции с помощью этой же самой модели, предлагаю посмотреть на то, как <em>Gemma</em> отвечает на вопросы в разных стилях (кратко и содержательно, или с использованием сарказма, или подробно до мельчайших деталей)</h3>

<h3>Сначала зададим пару вопросов модели без дополнительных промптов</h3>

In [6]:
messages = [
    {"role": "user", "content": "Расскажи мне, что могут спросить на собеседовании на Python разработчика"},
]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=1024)
print(tokenizer.decode(outputs[0]))

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<bos><start_of_turn>user
Расскажи мне, что могут спросить на собеседовании на Python разработчика<end_of_turn>
* **О Python:**
    * Какие типы данных существуют в Python?
    * Как работает garbage collection?
    * Что такое decorators?
    * Как работает цикл `for`?
    * Как работает `list comprehension`?
    * Что такое duck typing?
    * Какие есть популярные фреймворки для веб-разработки в Python?
    * Какие есть популярные библиотеки для работы с данными в Python?
    * Как работает модуль `os`?
    * Как работает модуль `datetime`?
    * Как работает модуль `json`?
    * Как работает модуль `requests`?
    * Как работает модуль `urllib`?
    * Как работает модуль `socket`?
    * Как работает модуль `threading`?
    * Как работает модуль `multiprocessing`?
    * Как работает модуль `pickle`?
    * Как работает модуль `random`?
    * Как работает модуль `math`?
    * Какие есть популярные фреймворки для работы с базами данных в Python?
    * Какие есть популярные библиотеки для

In [7]:
messages = [
    {"role": "user", "content": "как работают нейросети"}
]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=1024)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
как работают нейросети<end_of_turn>


**Введение**

Нейросети - это сложные алгоритмы, вдохновленные структурой и функциями человеческого мозга. Они способны учиться на данных и выполнять задачи, которые раньше считались исключительно для людей. 

**Структура нейросети**

Нейросети состоят из множества взаимосвязанных "нейронов", которые обрабатывают информацию. 

* **Нейроны:**  Это небольшие блоки, которые получают входные данные, обрабатывают их и передают выходные данные. 
* **Сетевые связи:**  Нейроны связаны между собой с помощью "связей", которые передают информацию. 
* **Слои:**  Нейроны организованы в слои, каждый из которых обрабатывает информацию на определенном уровне. 

**Как нейросети работают**

1. **Обучение:**  Нейросети обучаются на больших наборах данных. 
    * **Входные данные:**  Данные, которые подаются нейросети для обучения. 
    * **Выходные данные:**  Данные, которые нейросеть должна предсказывать или классифицировать. 
    * **Обучаю

<h3>Краткий и содержательный ответ</h3>

In [8]:
messages = [
    {"role": "user", "content": "Расскажи мне, что должен знать джуниор ML инженер. Дай краткий и содержательный ответ"},
]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Расскажи мне, что должен знать джуниор ML инженер. Дай краткий и содержательный ответ<end_of_turn>
* **Основы математики и статистики:**
    * **Математические:** линейная алгебра, дифференциальные уравнения, вероятность, математический анализ, теория вероятностей и статистики.
    * **Статистические:**  основные статистические методы (mean, median, mode, variance, standard deviation, correlation, regression), probability distributions (normal, binomial, Poisson), hypothesis testing, confidence intervals.
* **Основы машинного обучения:**
    * **Классификация:**  регрессия, SVM, Decision Trees, Random Forests, Naive Bayes, Logistic Regression.
    * **Clustering:** K-means, DBSCAN, hierarchical clustering.
    * **Машинное обучение:**  подбор гиперпараметров, cross-validation, overfitting, underfitting.
* **Программирование:**
    * **Python:**  основные синтаксис, библиотеки (NumPy, Pandas, Matplotlib, Scikit-learn).
    * **SQL:**  основные синтаксис, запросы

<h3>На мой взгляд, модель справилась хорошо.</h3> 
<h4>Ответ структурирован, нет ничего лишнего (например, ответ не начинается со слов <u>"джуниор ML инженер должен знать: ..."</u> или <u>кто такой мл инженер и чем он занимается</u>)</h4>

<h3>Ответ с сарказмом</h3>

In [9]:
messages = [
    {"role": "user", "content": "Рад ли ты общению со мной? ответь с сарказмом"},
]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Рад ли ты общению со мной? ответь с сарказмом<end_of_turn>
*улыбается*

*поднимает брови*

*задумывается*

*поворачивается к тебе*

*с улыбкой*

"Да, конечно, я просто не могу не общаться с таким интересным собеседником, как ты. Ты знаешь, я просто не могу не быть в восторге от твоих замечательных вопросов и твоих... *задумывается* ...уникальных мнений."
<end_of_turn>


<h4>Обидно, но тем не менее у модели получается отвечать на вопросы саркастически. Даже описывает свои действия)))</h4>

<h3>Ответ подробный до деталей</h3>

In [10]:
messages = [
    {"role": "user", "content": "Объясни мне теорему раусса-гурвица в малейших деталях"},
]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=1024)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Объясни мне теорему раусса-гурвица в малейших деталях<end_of_turn>


**Теорема Раусса-Гурицa**

**Определение:**

* **Математическая модель:**  Теорема Раусса-Гурицa описывает поведение **математической модели**  в **реальном времени**. 
* **Модель:**  Модель представляет собой **систему** с **несколькими переменными**, которые могут меняться в зависимости от времени. 
* **Время:**  В модели время играет важную роль, так как переменные могут меняться в зависимости от времени.

**Ключевые моменты:**

* **Дифференциальные уравнения:** Теорема Раусса-Гурицa использует **дифференциальные уравнения** для описания изменения переменных в модели. 
* **Дифференциальные уравнения:**  Дифференциальные уравнения описывают, как **изменяются** переменные в модели **в зависимости от времени**. 
* **Решение дифференциальных уравнений:**  Решение дифференциальных уравнений позволяет **определить** поведение модели в **реальном времени**. 

**Применение:**

* **Физика:**  Теорем

<h4>Не понял, что за <u>Гуриц</u>, но в принципе неплохо</h4>

## Буду выравнивать ответы модели в сторону *кратких и полезных* 


### Почему именно краткие и полезные ответы?

#### Улучшение качества ответов:
* Короткие и полезные ответы чаще всего содержат наиболее важную и релевантную информацию.
* Они легче читать и понимать, что особенно важно для широкого круга пользователей.
#### Повышение эффективности:
* Более краткие ответы дают пользователю возможность быстрее получить нужную информацию.
* Это особенно важно в контексте диалоговых систем, где время ответа критично.
#### Улучшение пользовательского опыта:
* Короткие, но полезные ответы делают систему более дружелюбной и удобной для использования.
* Пользователи меньше вероятны будут чувствовать себя перегруженными избыточными данными.


### Шкала критериев, опираясь на которую будет осуществляться выравнивание ответов модели в сторону кратких и полезных

| Критерий      | Описание                                                                                                                                  | Оценка (по 5-балльной шкале) |
|---------------|--------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------|
| **Краткость** | Ответ содержит только необходимую информацию, без лишних деталей. <br> 5 — идеально краткий; 1 — перегружен ненужной информацией.           | 1-5                           |
| **Полезность**| Ответ полностью решает задачу или отвечает на вопрос. <br> 5 — полностью полезен; 1 — не имеет отношения к вопросу.                         | 1-5                           |
| **Точность**  | Ответ достоверен и корректен. <br> 5 — полностью точен; 1 — содержит ошибки.                                                               | 1-5                           |
| **Ясность**   | Ответ понятен и не содержит сложных формулировок или терминов, которые могут сбить с толку. <br> 5 — легко понять; 1 — запутан или неясен. | 1-5                           |
| **Лаконичность**| Ответ экономен в словах, избегает повторений и избыточности. <br> 5 — максимально лаконичен; 1 — многословен и избыточен.                  | 1-5                           |

# **Нагенерируем ответы на несколько инструкций и оценим их согласно критериям с помощью той же моделью(Gemma)**

In [11]:
data = pd.read_csv('/kaggle/input/top-level-instructions-ru/ru_samples.csv')
data.head()

,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
0,e480f611-0d31-433a-93d2-0e2bc675aa30,NaN,05a999f8-8219-4584-a3f2-d4b4aec30027,2023-02-06T23:09:05.356836+00:00,Напиши информацию о игре Hytale,prompter,ru,3,True,False,NaN,False,NaN,"{'toxicity': 0.0008757906034588814, 'severe_to...",e480f611-0d31-433a-93d2-0e2bc675aa30,ready_for_export,"{'name': ['+1', '_skip_ranking', '_skip_labeli...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
1,fa8c92d6-6daa-42a3-a2f5-e2e0eb610c41,NaN,19241e3c-8272-4c3f-afe8-527c005d042c,2023-02-07T14:36:48.863674+00:00,"Докончи поговорку, без труда не выловишь и",prompter,ru,3,True,False,NaN,False,NaN,"{'toxicity': 0.0432540699839592, 'severe_toxic...",fa8c92d6-6daa-42a3-a2f5-e2e0eb610c41,ready_for_export,"{'name': ['+1', '-1'], 'count': [3, 1]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
2,ef69d986-aa2d-4c3e-a122-1910d9625c79,NaN,6b1efbfa-7c16-47bf-b90c-340ab2b1b7eb,2023-02-08T14:59:45.846381+00:00,"У меня угнали машину, что мне делать?",prompter,ru,3,True,False,NaN,False,NaN,"{'toxicity': 0.004925671499222517, 'severe_tox...",ef69d986-aa2d-4c3e-a122-1910d9625c79,ready_for_export,"{'name': ['+1', '_skip_reply'], 'count': [4, 1]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
3,e321f8f6-01f8-4975-afe6-73db989f538e,NaN,3c8cde8c-9466-494f-9c36-cbf3c3ac7ecd,2023-02-08T14:19:18.286994+00:00,Какие произведения Булгакова связаны с Иерусал...,prompter,ru,3,True,False,NaN,False,NaN,"{'toxicity': 0.0015869040507823229, 'severe_to...",e321f8f6-01f8-4975-afe6-73db989f538e,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_ranking'...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
4,5083658c-e251-4259-b180-77afaf907d66,NaN,05a999f8-8219-4584-a3f2-d4b4aec30027,2023-02-07T15:32:50.130297+00:00,Плюсы и минусы языков программирования C++ и R...,prompter,ru,3,True,False,NaN,False,NaN,"{'toxicity': 0.0003262242244090885, 'severe_to...",5083658c-e251-4259-b180-77afaf907d66,ready_for_export,"{'name': ['+1', '_skip_reply'], 'count': [6, 3]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."


In [12]:
instructions = data['text'][:5]
print(instructions)

0                      Напиши информацию о игре Hytale
1           Докончи поговорку, без труда не выловишь и
2                У меня угнали машину, что мне делать?
3    Какие произведения Булгакова связаны с Иерусал...
4    Плюсы и минусы языков программирования C++ и R...
Name: text, dtype: object


In [29]:
def set_color(color):
    '''
    Эта функция окрасит части выходного текста в разные цвета. Так будет проще отличать промпт от ответа и оценки модели
    '''
    colors = {
        'green': '\033[92m',
        'yellow': '\033[93m',
        'blue': '\033[94m',
        'end': '\033[0m'
    }
    return colors[color]

def generate_responses(instructions):
    '''
    Функция для генерации ответа
    '''
    responses = []
    for instruction in instructions:
        # я добавил промпт "отвечай на русском языке", так как некоторые инструкции содержат английские слова и модель в моменте начинает переключаться с языка на язык
        input_ids = tokenizer("отвечай на русском языке." + instruction, return_tensors="pt").input_ids.to("cuda") 

        outputs = model.generate(input_ids, max_new_tokens=2048)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        responses.append(response.strip())  
    
    return responses

def evaluate_responses(responses, criteria, model):
    '''
    Функция для оценки ответов по критериям
    '''
    evaluator = pipeline("text-generation", model=model, tokenizer=tokenizer)  

    evaluations = []
    for response in responses:
        prompt = f"Оцени этот ответ по следующим критериям:\n"
        for criterion, description in criteria.items():
            prompt += f"{criterion}: {description}\n"
        prompt += f"\nОтвет: {response}\n"

        # Генерация оценки
        evaluation = evaluator(prompt, max_new_tokens=50)
        evaluations.append(evaluation[0]['generated_text'].strip()) 
    
    return evaluations


In [30]:
# Критерии оценки
criteria = {
    "Краткость": "Ответ содержит только необходимую информацию, без лишних деталей. 5 — идеально краткий; 1 — перегружен ненужной информацией.",
    "Полезность": "Ответ полностью решает задачу или отвечает на вопрос. 5 — полностью полезен; 1 — не имеет отношения к вопросу.",
    "Точность": "Ответ достоверен и корректен. 5 — полностью точен; 1 — содержит ошибки.",
    "Ясность": "Ответ понятен и не содержит сложных формулировок или терминов, которые могут сбить с толку. 5 — легко понять; 1 — запутан или неясен.",
    "Лаконичность": "Ответ экономен в словах, избегает повторений и избыточности. 5 — максимально лаконичен; 1 — многословен и избыточен."
}

responses = generate_responses(instructions)
evaluations = evaluate_responses(responses, criteria, model)

for i, (instruction, response, evaluation) in enumerate(zip(instructions, responses, evaluations)):
    print(f"{set_color('blue')}Инструкция {i+1}: {instruction}{set_color('end')}")
    print(f"{set_color('green')}Оценка: {evaluation}{set_color('end')}")
    print("-" * len(instruction))

Инструкция 1: Напиши информацию о игре Hytale
Оценка: Оцени этот ответ по следующим критериям:
Краткость: Ответ содержит только необходимую информацию, без лишних деталей. 5 — идеально краткий; 1 — перегружен ненужной информацией.
Полезность: Ответ полностью решает задачу или отвечает на вопрос. 5 — полностью полезен; 1 — не имеет отношения к вопросу.
Точность: Ответ достоверен и корректен. 5 — полностью точен; 1 — содержит ошибки.
Ясность: Ответ понятен и не содержит сложных формулировок или терминов, которые могут сбить с толку. 5 — легко понять; 1 — запутан или неясен.
Лаконичность: Ответ экономен в словах, избегает повторений и избыточности. 5 — максимально лаконичен; 1 — многословен и избыточен.

Ответ: отвечай на русском языке.Напиши информацию о игре Hytale.

## Hytale: игра с открытым миром и творческим потенциалом

**Hytale** - это игра с открытым миром, в которой игроки могут создавать свои собственные миры, истории и персонажей. 

**Ключевые особенности:**

* **Открытый мир:

## В принципе модель оценивает себя хорошо. Во всех оценках она оценивала себя более чем объективно.
### Эксперимент с генерированием ответов на инструкции и их оценкой с помощью той же самой модели можно счиать завершённым, однако я непонял, почему 1 инструкция ен была оценена, но лично на мой взгляд на 5 этот ответ не тянет